In [ ]:
import time
import random
from seal import EncryptionParameters, scheme_type, \
    SEALContext, print_parameters, KeyGenerator, \
    Encryptor, CoeffModulus, Evaluator, Decryptor, \
    Plaintext, Ciphertext

In [ ]:
print("Example: BFV Basics");

In this example, we demonstrate performing simple computations (a polynomial<br>
evaluation) on encrypted integers using the BFV encryption scheme.<br>
<br>
The first task is to set up an instance of the EncryptionParameters class.<br>
It is critical to understand how the different parameters behave, how they<br>
affect the encryption scheme, performance, and the security level. There are<br>
three encryption parameters that are necessary to set:<br>
<br>
    - poly_modulus_degree (degree of polynomial modulus);<br>
    - coeff_modulus ([ciphertext] coefficient modulus);<br>
    - plain_modulus (plaintext modulus; only for the BFV scheme).<br>
<br>
The BFV scheme cannot perform arbitrary computations on encrypted data.<br>
Instead, each ciphertext has a specific quantity called the `invariant noise<br>
budget' -- or `noise budget' for short -- measured in bits. The noise budget<br>
in a freshly encrypted ciphertext (initial noise budget) is determined by<br>
the encryption parameters. Homomorphic operations consume the noise budget<br>
at a rate also determined by the encryption parameters. In BFV the two basic<br>
operations allowed on encrypted data are additions and multiplications, of<br>
which additions can generally be thought of as being nearly free in terms of<br>
noise budget consumption compared to multiplications. Since noise budget<br>
consumption compounds in sequential multiplications, the most significant<br>
factor in choosing appropriate encryption parameters is the multiplicative<br>
depth of the arithmetic circuit that the user wants to evaluate on encrypted<br>
data. Once the noise budget of a ciphertext reaches zero it becomes too<br>
corrupted to be decrypted. Thus, it is essential to choose the parameters to<br>
be large enough to support the desired computation; otherwise the result is<br>
impossible to make sense of even with the secret key.

In [ ]:
parms = EncryptionParameters(scheme_type.BFV)

The first parameter we set is the degree of the `polynomial modulus'. This<br>
must be a positive power of 2, representing the degree of a power-of-two<br>
cyclotomic polynomial; it is not necessary to understand what this means.<br>
<br>
Larger poly_modulus_degree makes ciphertext sizes larger and all operations<br>
slower, but enables more complicated encrypted computations. Recommended<br>
values are 1024, 2048, 4096, 8192, 16384, 32768, but it is also possible<br>
to go beyond this range.<br>
<br>
In this example we use a relatively small polynomial modulus. Anything<br>
smaller than this will enable only very restricted encrypted computations.

In [ ]:
poly_modulus_degree = 4096
parms.set_poly_modulus_degree(poly_modulus_degree)

Next we set the [ciphertext] `coefficient modulus' (coeff_modulus). This<br>
parameter is a large integer, which is a product of distinct prime numbers,<br>
each up to 60 bits in size. It is represented as a vector of these prime<br>
numbers, each represented by an instance of the SmallModulus class. The<br>
bit-length of coeff_modulus means the sum of the bit-lengths of its prime<br>
factors.<br>
<br>
A larger coeff_modulus implies a larger noise budget, hence more encrypted<br>
computation capabilities. However, an upper bound for the total bit-length<br>
of the coeff_modulus is determined by the poly_modulus_degree, as follows:<br>
<br>
    +----------------------------------------------------+<br>
    | poly_modulus_degree | max coeff_modulus bit-length |<br>
    +---------------------+------------------------------+<br>
    | 1024                | 27                           |<br>
    | 2048                | 54                           |<br>
    | 4096                | 109                          |<br>
    | 8192                | 218                          |<br>
    | 16384               | 438                          |<br>
    | 32768               | 881                          |<br>
    +---------------------+------------------------------+<br>
<br>
These numbers can also be found in native/src/seal/util/hestdparms.h encoded<br>
in the function SEAL_HE_STD_PARMS_128_TC, and can also be obtained from the<br>
function<br>
<br>
    CoeffModulus::MaxBitCount(poly_modulus_degree).<br>
<br>
For example, if poly_modulus_degree is 4096, the coeff_modulus could consist<br>
of three 36-bit primes (108 bits).<br>
<br>
Microsoft SEAL comes with helper functions for selecting the coeff_modulus.<br>
For new users the easiest way is to simply use<br>
<br>
    CoeffModulus::BFVDefault(poly_modulus_degree),<br>
<br>
which returns std::vector<SmallModulus> consisting of a generally good choice<br>
for the given poly_modulus_degree.

In [ ]:
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))

The plaintext modulus can be any positive integer, even though here we take<br>
it to be a power of two. In fact, in many cases one might instead want it<br>
to be a prime number; we will see this in later examples. The plaintext<br>
modulus determines the size of the plaintext data type and the consumption<br>
of noise budget in multiplications. Thus, it is essential to try to keep the<br>
plaintext data type as small as possible for best performance. The noise<br>
budget in a freshly encrypted ciphertext is<br>
<br>
    ~ log2(coeff_modulus/plain_modulus) (bits)<br>
<br>
and the noise budget consumption in a homomorphic multiplication is of the<br>
form log2(plain_modulus) + (other terms). <br>
<br>
The plaintext modulus is specific to the BFV scheme, and cannot be set when<br>
using the CKKS scheme.

In [ ]:
parms.set_plain_modulus(1024)

Now that all parameters are set, we are ready to construct a SEALContext<br>
object. This is a heavy class that checks the validity and properties of the<br>
parameters we just set.

In [ ]:
context = SEALContext.Create(parms)

Print the parameters that we have chosen.

In [ ]:
print("Set encryption parameters and print")
print_parameters(context) # TODO: FIX
print("~~~~~~ A naive way to calculate 4(x^2+1)(x+1)^2. ~~~~~~")

The encryption schemes in Microsoft SEAL are public key encryption schemes.<br>
For users unfamiliar with this terminology, a public key encryption scheme<br>
has a separate public key for encrypting data, and a separate secret key for<br>
decrypting data. This way multiple parties can encrypt data using the same<br>
shared public key, but only the proper recipient of the data can decrypt it<br>
with the secret key.<br>
<br>
We are now ready to generate the secret and public keys. For this purpose<br>
we need an instance of the KeyGenerator class. Constructing a KeyGenerator<br>
automatically generates the public and secret key, which can immediately be<br>
read to local variables.

In [ ]:
keygen = KeyGenerator(context)
public_key = keygen.public_key()
secret_key = keygen.secret_key()

To be able to encrypt we need to construct an instance of Encryptor. Note<br>
that the Encryptor only requires the public key, as expected.

In [ ]:
encryptor = Encryptor(context, public_key)

Computations on the ciphertexts are performed with the Evaluator class. In<br>
a real use-case the Evaluator would not be constructed by the same party<br>
that holds the secret key.

In [ ]:
evaluator = Evaluator(context)

We will of course want to decrypt our results to verify that everything worked,<br>
so we need to also construct an instance of Decryptor. Note that the Decryptor<br>
requires the secret key.

In [ ]:
decryptor = Decryptor(context, secret_key)

As an example, we evaluate the degree 4 polynomial<br>
<br>
    4x^4 + 8x^3 + 8x^2 + 8x + 4<br>
<br>
over an encrypted x = 6. The coefficients of the polynomial can be considered<br>
as plaintext inputs, as we will see below. The computation is done modulo the<br>
plain_modulus 1024.<br>
 <br>
While this examples is simple and easy to understand, it does not have much<br>
practical value. In later examples we will demonstrate how to compute more<br>
efficiently on encrypted integers and real or complex numbers.<br>
<br>
Plaintexts in the BFV scheme are polynomials of degree less than the degree<br>
of the polynomial modulus, and coefficients integers modulo the plaintext<br>
modulus. For readers with background in ring theory, the plaintext space is<br>
the polynomial quotient ring Z_T[X]/(X^N + 1), where N is poly_modulus_degree<br>
and T is plain_modulus.<br>
<br>
To get started, we create a plaintext containing the constant 6. For the<br>
plaintext element we use a constructor that takes the desired polynomial as<br>
a string with coefficients represented as hexadecimal numbers.

In [ ]:
x = 6
x_plain = Plaintext(str(x))
print("Express x = {} as a plaintext polynomial 0x{}.".format(x, x_plain.to_string()))

We then encrypt the plaintext, producing a ciphertext.

In [ ]:
x_encrypted = Ciphertext()
print("Encrypt x_plain to x_encrypted.")
encryptor.encrypt(x_plain, x_encrypted)

In Microsoft SEAL, a valid ciphertext consists of two or more polynomials<br>
whose coefficients are integers modulo the product of the primes in the<br>
coeff_modulus. The number of polynomials in a ciphertext is called its `size'<br>
and is given by Ciphertext::size(). A freshly encrypted ciphertext always<br>
has size 2.

In [ ]:
print("    + size of freshly encrypted x: {}".format(x_encrypted.size()))

There is plenty of noise budget left in this freshly encrypted ciphertext.

In [ ]:
print("    + noise budget in freshly encrypted x: {} bits".format(
      decryptor.invariant_noise_budget(x_encrypted)))

We decrypt the ciphertext and print the resulting plaintext in order to<br>
demonstrate correctness of the encryption.

In [ ]:
x_decrypted = Plaintext()
decryptor.decrypt(x_encrypted, x_decrypted)
print("    + decryption of x_encrypted: 0x{} ...... Correct.".format(x_decrypted.to_string()))

When using Microsoft SEAL, it is typically advantageous to compute in a way<br>
that minimizes the longest chain of sequential multiplications. In other<br>
words, encrypted computations are best evaluated in a way that minimizes<br>
the multiplicative depth of the computation, because the total noise budget<br>
consumption is proportional to the multiplicative depth. For example, for<br>
our example computation it is advantageous to factorize the polynomial as<br>
<br>
    4x^4 + 8x^3 + 8x^2 + 8x + 4 = 4(x + 1)^2 * (x^2 + 1)<br>
<br>
to obtain a simple depth 2 representation. Thus, we compute (x + 1)^2 and<br>
(x^2 + 1) separately, before multiplying them, and multiplying by 4.<br>
<br>
First, we compute x^2 and add a plaintext "1". We can clearly see from the<br>
print-out that multiplication has consumed a lot of noise budget. The user<br>
can vary the plain_modulus parameter to see its effect on the rate of noise<br>
budget consumption.

In [ ]:
print("Compute x_sq_plus_one (x^2+1).")
x_sq_plus_one = Ciphertext()
evaluator.square(x_encrypted, x_sq_plus_one);
plain_one = Plaintext("1");
evaluator.add_plain_inplace(x_sq_plus_one, plain_one)

Encrypted multiplication results in the output ciphertext growing in size.<br>
More precisely, if the input ciphertexts have size M and N, then the output<br>
ciphertext after homomorphic multiplication will have size M+N-1. In this<br>
case we perform a squaring, and observe both size growth and noise budget<br>
consumption.

In [ ]:
print("    + size of x_sq_plus_one: {}".format(x_sq_plus_one.size()))
print("    + noise budget in x_sq_plus_one: {} bits".format(
    decryptor.invariant_noise_budget(x_sq_plus_one)))

Even though the size has grown, decryption works as usual as long as noise<br>
budget has not reached 0.

In [ ]:
decrypted_result = Plaintext()
decryptor.decrypt(x_sq_plus_one, decrypted_result);
print("    + decryption of x_sq_plus_one: 0x{} ...... Correct.".format(decrypted_result.to_string()))

Next, we compute (x + 1)^2.

In [ ]:
print("Compute x_plus_one_sq ((x+1)^2).")
x_plus_one_sq = Ciphertext()
evaluator.add_plain(x_encrypted, plain_one, x_plus_one_sq);
evaluator.square_inplace(x_plus_one_sq);
print("    + size of x_plus_one_sq: {}".format(x_plus_one_sq.size()))
print("    + noise budget in x_plus_one_sq: {} bits".format(
    decryptor.invariant_noise_budget(x_plus_one_sq)))
decryptor.decrypt(x_plus_one_sq, decrypted_result)
print("    + decryption of x_plus_one_sq: 0x{} ...... Correct.".format(decrypted_result.to_string()))

Finally, we multiply (x^2 + 1) * (x + 1)^2 * 4.

In [ ]:
print("Compute encrypted_result (4(x^2+1)(x+1)^2).")
encrypted_result = Ciphertext()
plain_four = Plaintext("4")
evaluator.multiply_plain_inplace(x_sq_plus_one, plain_four)
evaluator.multiply(x_sq_plus_one, x_plus_one_sq, encrypted_result)
print("    + size of encrypted_result: {}".format(encrypted_result.size()))
print("    + noise budget in encrypted_result: {} bits".format(
    decryptor.invariant_noise_budget(encrypted_result)))
print("NOTE: Decryption can be incorrect if noise budget is zero.")
print("~~~~~~ A better way to calculate 4(x^2+1)(x+1)^2. ~~~~~~")

Noise budget has reached 0, which means that decryption cannot be expected<br>
to give the correct result. This is because both ciphertexts x_sq_plus_one<br>
and x_plus_one_sq consist of 3 polynomials due to the previous squaring<br>
operations, and homomorphic operations on large ciphertexts consume much more<br>
noise budget than computations on small ciphertexts. Computing on smaller<br>
ciphertexts is also computationally significantly cheaper.

`Relinearization' is an operation that reduces the size of a ciphertext after<br>
multiplication back to the initial size, 2. Thus, relinearizing one or both<br>
input ciphertexts before the next multiplication can have a huge positive<br>
impact on both noise growth and performance, even though relinearization has<br>
a significant computational cost itself. It is only possible to relinearize<br>
size 3 ciphertexts down to size 2, so often the user would want to relinearize<br>
after each multiplication to keep the ciphertext sizes at 2.

Relinearization requires special `relinearization keys', which can be thought<br>
of as a kind of public key. Relinearization keys can easily be created with<br>
the KeyGenerator.

Relinearization is used similarly in both the BFV and the CKKS schemes, but<br>
in this example we continue using BFV. We repeat our computation from before,<br>
but this time relinearize after every multiplication.

We use KeyGenerator::relin_keys() to create relinearization keys.

In [ ]:
print("Generate relinearization keys.")
relin_keys = keygen.relin_keys();

We now repeat the computation relinearizing after each multiplication.

In [ ]:
print("Compute and relinearize x_squared (x^2),")
print("then compute x_sq_plus_one (x^2+1)")
x_squared = Ciphertext()
evaluator.square(x_encrypted, x_squared);
print("    + size of x_squared: {}" .format(x_squared.size()))
evaluator.relinearize_inplace(x_squared, relin_keys);
print("    + size of x_squared (after relinearization): {}".format(
    x_squared.size()))
evaluator.add_plain(x_squared, plain_one, x_sq_plus_one)
print("    + noise budget in x_sq_plus_one: {} bits".format(
    decryptor.invariant_noise_budget(x_sq_plus_one))) 
decryptor.decrypt(x_sq_plus_one, decrypted_result)
print("    + decryption of x_sq_plus_one: 0x{}  ...... Correct.".format(decrypted_result.to_string()))

In [ ]:
x_plus_one = Ciphertext()
print("Compute x_plus_one (x+1),")
print("then compute and relinearize x_plus_one_sq ((x+1)^2).")
evaluator.add_plain(x_encrypted, plain_one, x_plus_one)
evaluator.square(x_plus_one, x_plus_one_sq)
print("    + size of x_plus_one_sq: {}".format(x_plus_one_sq.size()))
evaluator.relinearize_inplace(x_plus_one_sq, relin_keys)
print("    + noise budget in x_plus_one_sq: {} bits".format(
    decryptor.invariant_noise_budget(x_plus_one_sq)) )
decryptor.decrypt(x_plus_one_sq, decrypted_result)
print("    + decryption of x_plus_one_sq: 0x{}  ...... Correct.".format(decrypted_result.to_string()))

In [ ]:
print("Compute and relinearize encrypted_result (4(x^2+1)(x+1)^2).")
evaluator.multiply_plain_inplace(x_sq_plus_one, plain_four)
evaluator.multiply(x_sq_plus_one, x_plus_one_sq, encrypted_result)
print("    + size of encrypted_result: {}".format(encrypted_result.size()))
evaluator.relinearize_inplace(encrypted_result, relin_keys)
print("    + size of encrypted_result (after relinearization): {}".format(
    encrypted_result.size()))
print("    + noise budget in encrypted_result: {} bits".format(
    decryptor.invariant_noise_budget(encrypted_result)))

In [ ]:
print("NOTE: Notice the increase in remaining noise budget.")

Relinearization clearly improved our noise consumption. We have still plenty<br>
of noise budget left, so we can expect the correct answer when decrypting.

In [ ]:
print("Decrypt encrypted_result (4(x^2+1)(x+1)^2).")
decryptor.decrypt(encrypted_result, decrypted_result)
print("    + decryption of 4(x^2+1)(x+1)^2 = 0x{}  ...... Correct.".format(
    decrypted_result.to_string()))

For x=6, 4(x^2+1)(x+1)^2 = 7252. Since the plaintext modulus is set to 1024,<br>
this result is computed in integers modulo 1024. Therefore the expected output<br>
should be 7252 % 1024 == 84, or 0x54 in hexadecimal.